import re
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
# Imports

In [254]:
# pip install dijkstar

In [255]:
import folium
from collections import Counter
import postman_problems
from postman_problems.solver import cpp
from postman_problems.stats import calculate_postman_solution_stats
import networkx as nx
import itertools
import dwave_networkx as dnx
import dimod
import time
import dijkstar

## My Data

In [256]:
stations_df = pd.read_csv("./saved_data/final_station_df.csv", index_col = 0)

<IPython.core.display.Javascript object>

In [257]:
non_unique_stations_df = pd.read_csv("./saved_data/non_unique_mta_stations.csv", index_col=0)

<IPython.core.display.Javascript object>

In [258]:
my_edgelist = pd.read_csv('./saved_data/edge_list_df_no_req.csv', index_col=0)

<IPython.core.display.Javascript object>

In [259]:
node_list_df = pd.read_csv("./saved_data/nodelist_nyc_subway.csv", index_col=0).reset_index()

<IPython.core.display.Javascript object>

In [260]:
node_list_df

,station_id,X,Y
0,101,40.889248,-73.898583
1,103,40.884667,-73.900870
2,104,40.878856,-73.904834
3,106,40.874561,-73.909831
4,107,40.869444,-73.915279
...,...,...,...
446,R43,40.629742,-74.025510
447,R44,40.622687,-74.028398
448,S01,40.680596,-73.955827
449,S03,40.674772,-73.957624


# Mini Dijkstar Attempt

## Getting A,C,E Edges

In [154]:
good_indices_ace = []
for idx, x in enumerate(my_edgelist['node1']):
    for letter in 'A':
        if letter in x and letter in my_edgelist['node2'][idx]:
            if idx not in good_indices_ace:
                good_indices_ace.append(idx)

In [155]:
A_edgelist = my_edgelist.iloc[good_indices_ace]

In [66]:
AC_edgelist.head(2)

,node1,node2,trail,color,distance
194,A02,A03,nyc subway,red,90
195,A03,A05,nyc subway,red,120


In [156]:
A_edgelist.reset_index(inplace=True, drop=True)

In [157]:
A_edgelist[A_edgelist['node1'] == '112_A09']

,node1,node2,trail,color,distance
5,112_A09,A12_D13,nyc subway,red,210
6,112_A09,A10,nyc subway,red,90


## Getting A,C,E Nodes

In [150]:
A_nodelist = node_list_df[node_list_df.station_id.str.contains("A")]
# C_nodelist = node_list_df[node_list_df.station_id.str.contains("C")]
# E_nodelist = node_list_df[node_list_df.station_id.str.contains("E")]
# A_nodelist = pd.concat([A_nodelist, C_nodelist])

In [172]:
A_nodelist.reset_index(inplace=True, drop=True)

In [151]:
A_nodelist.shape

(52, 3)

In [167]:
A_nodelist.head()

,station_id,X,Y
9,112_A09,40.840556,-73.940133
22,125_A24,40.768247,-73.981929
174,A02,40.868072,-73.919899
175,A03,40.865491,-73.927271
176,A05,40.859022,-73.934180


## Making TSP Matrix

In [152]:
dijk_ace_graph = dijkstar.Graph(undirected=True)

In [158]:
zipped_edges = list(zip(A_edgelist['node1'], A_edgelist['node2'], A_edgelist['distance']))
for x in zipped_edges:
    dijk_ace_graph.add_edge(x[0], x[1], x[2])

In [159]:
len(zipped_edges)

63

### Find distances between all pairs

In [183]:
def get_indirect_distance(dijkstar_graph, nodelist):
    distance_matrix = []
    for i in range(len(nodelist)):
        start_node = nodelist['station_id'][i]
        one_node_tree = []
        for num in range(i, len(nodelist)):
            dest_node = nodelist['station_id'][num]
            path = dijkstar.find_path(dijkstar_graph, start_node, dest_node)
            path_distance = path.total_cost
            one_node_tree.append(path_distance)
        distance_matrix.append(one_node_tree)  # top right triangle of matrix
    for i in range(len(distance_matrix)):
#         print(f"i = {i}")
        for x in range(i, len(distance_matrix)):
#             print(f"x = {x}")
            if distance_matrix[i][x] != 0:
                distance_matrix[x].insert(i, distance_matrix[i][x])
    return distance_matrix

In [184]:
A_dist_matrix = get_indirect_distance(dijk_ace_graph, A_nodelist)

In [185]:
len(A_dist_matrix)

52

In [186]:
len(A_dist_matrix[0])

52

In [187]:
len(A_dist_matrix[-1])

52

## Using OR Tools

**Note**: This code was adapted from the above python file developed by Google OR-Tools. License can be found here: https://www.apache.org/licenses/LICENSE-2.0

- This is trying to reach every stop and return to the original node

In [220]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(A_dist_matrix):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = A_dist_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(A_dist_matrix)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 9210 seconds
Route for vehicle 0:
 0 -> 7 -> 8 -> 9 -> 10 -> 11 -> 12 -> 13 -> 14 -> 15 -> 16 -> 17 -> 18 -> 1 -> 19 -> 20 -> 21 -> 22 -> 23 -> 24 -> 25 -> 26 -> 27 -> 28 -> 29 -> 30 -> 31 -> 32 -> 33 -> 34 -> 35 -> 36 -> 37 -> 38 -> 39 -> 40 -> 41 -> 42 -> 43 -> 44 -> 45 -> 46 -> 47 -> 48 -> 49 -> 50 -> 51 -> 6 -> 5 -> 4 -> 3 -> 2 -> 0



<IPython.core.display.Javascript object>

In [311]:
.13 * 60

7.800000000000001

In [329]:
def get_time_in_hrs(seconds):
    minutes = seconds / 60
    hours = minutes / 60
    remainder_hour = hours % 1
    final_minutes = round((remainder_hour * 60), 2)
    return f"The entire route takes {hours - remainder_hour} hours, {final_minutes} minutes"

### Translating list of stops to station_ids

In [224]:
def tsp_station_ids(list_of_stops, nodelist):
    nodes = nodelist['station_id']
    list_of_nodes = [nodes[int(x)] for x in list_of_stops]
    return list_of_nodes

In [225]:
x = tsp_station_ids(list_of_stops, A_nodelist)

In [227]:
# x

### Getting station names in order

In [380]:
non_unique_stations_df[non_unique_stations_df['stop_id'] == '101']

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,101,Van Cortlandt Park - 242 St,40.889248,-73.898583,1,NaN


In [381]:
non_unique_stations_df['stop_name'][0]

'Van Cortlandt Park - 242 St'

In [391]:
def get_station_names(stations_df, tsp_station_ids):
    full_list = []
    for mta_station in tsp_station_ids:
        single_stations = mta_station.split("_")
        station_string = ""
        counter = 0
        for station in single_stations:
            row = stations_df.index[stations_df['stop_id'] == station].tolist()
            for item in row:
                name = stations_df['stop_name'][item]
                if counter >= 1:
                    station_string += " / " + name
                    counter += 1
                elif counter == 0:
                    station_string += name
                    counter += 1
        full_list.append(station_string)
    return full_list

# Applying to All the Data

In [261]:
nyc_graph = dijkstar.Graph(undirected=True)

In [262]:
nyc_zipped_edges = list(zip(my_edgelist['node1'], my_edgelist['node2'], my_edgelist['distance']))
for x in nyc_zipped_edges:
    nyc_graph.add_edge(x[0], x[1], x[2])

In [263]:
nyc_dist_matrix = get_indirect_distance(nyc_graph, node_list_df)

In [269]:
len(nyc_dist_matrix)

451

## Running Algorithm

In [271]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(nyc_dist_matrix):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = nyc_dist_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_seconds = solution.ObjectiveValue()
    print('Objective: {} seconds'.format(total_seconds))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_seconds, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(nyc_dist_matrix)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        seconds, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return seconds, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    seconds, list_of_stops = main()
    
# [END program]

Objective: 76080 seconds
Route for vehicle 0:
 0 -> 1 -> 2 -> 3 -> 4 -> 5 -> 6 -> 7 -> 8 -> 9 -> 173 -> 174 -> 175 -> 176 -> 177 -> 10 -> 11 -> 12 -> 13 -> 14 -> 15 -> 16 -> 20 -> 19 -> 18 -> 17 -> 61 -> 60 -> 59 -> 58 -> 92 -> 91 -> 56 -> 55 -> 54 -> 53 -> 52 -> 51 -> 50 -> 49 -> 48 -> 47 -> 46 -> 45 -> 44 -> 43 -> 42 -> 41 -> 40 -> 39 -> 38 -> 114 -> 113 -> 112 -> 111 -> 110 -> 57 -> 105 -> 132 -> 131 -> 130 -> 129 -> 128 -> 127 -> 126 -> 125 -> 124 -> 123 -> 122 -> 121 -> 120 -> 119 -> 118 -> 117 -> 116 -> 115 -> 133 -> 134 -> 135 -> 136 -> 137 -> 138 -> 139 -> 140 -> 141 -> 142 -> 144 -> 145 -> 146 -> 80 -> 79 -> 78 -> 77 -> 76 -> 75 -> 74 -> 90 -> 89 -> 88 -> 87 -> 86 -> 85 -> 84 -> 83 -> 82 -> 81 -> 73 -> 72 -> 71 -> 70 -> 69 -> 37 -> 36 -> 35 -> 25 -> 26 -> 27 -> 28 -> 29 -> 30 -> 31 -> 32 -> 33 -> 34 -> 62 -> 63 -> 64 -> 65 -> 66 -> 67 -> 68 -> 109 -> 108 -> 107 -> 106 -> 152 -> 151 -> 150 -> 149 -> 148 -> 366 -> 194 -> 193 -> 192 -> 191 -> 190 -> 249 -> 286 -> 285 -> 284 -> 32

<IPython.core.display.Javascript object>

In [334]:
hours = get_time_in_hrs(seconds)
hours

'The entire route takes 21.0 hours, 8.0 minutes'

In [272]:
full_nyc_tsp_stations = tsp_station_ids(list_of_stops, node_list_df)

In [273]:
# full_nyc_tsp_stations

['101',
 '103',
 '104',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112_A09',
 'A02',
 'A03',
 'A05',
 'A06',
 'A07',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '123',
 '122',
 '121',
 '120',
 '227',
 '226',
 '225',
 '224',
 '302',
 '301',
 '221',
 '220',
 '219',
 '218',
 '217',
 '216',
 '215',
 '214',
 '213',
 '212',
 '211',
 '210',
 '209',
 '208',
 '207',
 '206',
 '205',
 '204',
 '201',
 '505',
 '504',
 '503',
 '502',
 '501',
 '222_415',
 '416',
 '619',
 '618',
 '617',
 '616',
 '615',
 '614',
 '613',
 '612',
 '611',
 '610',
 '609',
 '608',
 '607',
 '606',
 '604',
 '603',
 '602',
 '601',
 '621',
 '622',
 '623',
 '624',
 '625',
 '626',
 '627',
 '628',
 '629',
 '630',
 '632',
 '633',
 '634',
 '247',
 '246',
 '245',
 '244',
 '243',
 '242',
 '241',
 '257',
 '256',
 '255',
 '254',
 '253',
 '252',
 '251',
 '250',
 '249',
 '248',
 '239',
 '238',
 '237',
 '236',
 '235',
 '140_142',
 '139',
 '138',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '13

In [336]:
non_unique_stations_df[non_unique_stations_df['stop_id'].str.contains('M08')]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
1224,M08,Myrtle - Wyckoff Avs,40.69943,-73.912385,1,NaN


In [392]:
tsp_names = get_station_names(non_unique_stations_df, full_nyc_tsp_stations)

In [393]:
tsp_names

['Van Cortlandt Park - 242 St',
 '238 St',
 '231 St',
 'Marble Hill - 225 St',
 '215 St',
 '207 St',
 'Dyckman St',
 '191 St',
 '181 St',
 '168 St - Washington Hts / 168 St',
 'Inwood - 207 St',
 'Dyckman St',
 '190 St',
 '181 St',
 '175 St',
 '157 St',
 '145 St',
 '137 St - City College',
 '125 St',
 '116 St - Columbia University',
 'Cathedral Pkwy',
 '103 St',
 '72 St',
 '79 St',
 '86 St',
 '96 St',
 'Central Park North (110 St)',
 '116 St',
 '125 St',
 '135 St',
 '145 St',
 'Harlem - 148 St',
 '3 Av - 149 St',
 'Jackson Av',
 'Prospect Av',
 'Intervale Av',
 'Simpson St',
 'Freeman St',
 '174 St',
 'West Farms Sq - E Tremont Av',
 'E 180 St',
 'Bronx Park East',
 'Pelham Pkwy',
 'Allerton Av',
 'Burke Av',
 'Gun Hill Rd',
 '219 St',
 '225 St',
 '233 St',
 'Nereid Av',
 'Wakefield - 241 St',
 'Morris Park',
 'Pelham Pkwy',
 'Gun Hill Rd',
 'Baychester Av',
 'Eastchester - Dyre Av',
 '149 St - Grand Concourse / 149 St - Grand Concourse',
 '138 St - Grand Concourse',
 '3 Av - 138 St',


In [394]:
possibly_duplicate_stations = [k for k, v in Counter(tsp_names).items() if v > 1]

In [396]:
possibly_duplicate_stations.remove('Van Cortlandt Park - 242 St') # this should be here twice

In [399]:
possible_duplicates_in_df = []
for x in possibly_duplicate_stations:
    matching_row = non_unique_stations_df[non_unique_stations_df['stop_name'] == x]
    possible_duplicates_in_df.append(matching_row)

In [402]:
len(possible_duplicates_in_df)

63

In [403]:
possible_duplicates_in_df[0]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
18,109,Dyckman St,40.860531,-73.925536,1,NaN
540,A03,Dyckman St,40.865491,-73.927271,1,NaN


In [450]:
possible_duplicates_close_lat_lon = []
for x in possible_duplicates_in_df:
    lat_list = [round(lat, 3) for lat in x['stop_lat']]
    lon_list = [round(lon, 3) for lon in x['stop_lon']]
    if len(set(lat_list)) < len(lat_list) and len(set(lon_list)) < len(lon_list):
        possible_duplicates_close_lat_lon.append(x)
    

In [451]:
len(possible_duplicates_close_lat_lon)

3

In [452]:
possible_duplicates_close_lat_lon[2]

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
192,229,Fulton St,40.709416,-74.006571,1,NaN
324,418,Fulton St,40.710368,-74.009509,1,NaN
621,A38,Fulton St,40.710197,-74.007691,1,NaN
1041,G36,Fulton St,40.687119,-73.975375,1,NaN
1260,M22,Fulton St,40.710374,-74.007582,1,NaN
